# Trading algorithim

In [ ]:
# import all packages
import pandas as pd
import numpy as np
import degiroapi
import matplotlib.pyplot as plt
from degiroapi.product import Product
from degiroapi.order import Order
from degiroapi.utils import pretty_json
from datetime import datetime, timedelta

#Settings for graphs
%matplotlib inline
import matplotlib as mpl
import seaborn as sns
mpl.rcParams["figure.dpi"] = 400
mpl.rcParams["font.size"] = 8

In [ ]:
# Login To Degiro
degiro = degiroapi.DeGiro()
degiro.login("username", "password")

In [ ]:
portfolio = degiro.getdata(degiroapi.Data.Type.PORTFOLIO, True)
for element in portfolio:
    print(data)

In [ ]:
# Read in Transaction History
transactions = degiro.transactions(datetime(2019, 1, 1), datetime.now())

In [ ]:
# Calculate the key income and outgo from the portfolio for overview purposes
Purchase_value_local = 0
Purchase_value_eur = 0
Fee_paid = 0
Sell_value_local = 0
Sell_value_eur = 0
Fx_rate = 0
buysell_indicator = ""
number_of_shares = 0

for element in transactions:
    
    buysell_indicator = element["buysell"]
    number_of_shares = element["quantity"]
    
    if(element["totalInBaseCurrency"] == element["total"]):
        Fx_rate = 1
    else:
        Fx_rate = element["fxRate"]
        
    if(buysell_indicator == "B"):
        Purchase_value_local = element["price"] * number_of_shares + Purchase_value_local
        Purchase_value_eur   = ((element["price"] * number_of_shares) / Fx_rate) + Purchase_value_eur 
        
    elif(buysell_indicator == "S"):
        Sell_value_local = element["price"] * -number_of_shares + Sell_value_local
        Sell_value_eur   = ((element["price"] * -number_of_shares)/Fx_rate) + Sell_value_eur
        
    else:
        print("Unexpected value for buysell indicator" , buysell_indicator)
    
    Fee_paid = -element["feeInBaseCurrency"] + Fee_paid

Total_Profit = Sell_value_eur - Purchase_value_eur - Fee_paid

In [ ]:
# Graph the key outgoes for visual representation
axes = plt.axes()
wdth = 0.35
plt.bar("Bought (Local)",Purchase_value_local, wdth, color = "black", alpha = 0.1)
plt.bar("Sold (Local)", Sell_value_local, width, color = "black", alpha = 0.4)
plt.bar("Bought (EUR)", Purchase_value_eur, wdth, color = "green", alpha = 0.1)
plt.bar("Sold (EUR)", Sell_value_eur, wdth, color = "green", alpha = 0.4)
plt.bar("Fees", Fee_paid, wdth, color = "blue", alpha = 0.4)
plt.bar("Profit (Eur)", Total_Profit, wdth, color = "m", alpha = 1)
axes.yaxis.grid("on")

In [ ]:
# Get sample market information
product = degiro.search_products("pfizer")
product
info = degiro.product_info(product[0]["id"])
info
realprice = degiro.real_time_price(Product(product[0]).id, degiroapi.Interval.Type.One_Day)
realprice

In [ ]:
#Log Out of Degiro
degiro.logout()